Most of the code her is taken from this repository (https://github.com/tkipf/pygcn/tree/master/pygcn), from one of the original authors of the paper: 
```
@article{kipf2016semi,
  title={Semi-Supervised Classification with Graph Convolutional Networks},
  author={Kipf, Thomas N and Welling, Max},
  journal={arXiv preprint arXiv:1609.02907},
  year={2016}
}
```

In [1]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [2]:
import torch.nn as nn
import torch.nn.functional as F
# from pygcn.layers import GraphConvolution


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [3]:
import numpy as np
import scipy.sparse as sp
import torch


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="../data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [4]:
import time
import numpy as np
import torch.optim as optim

In [5]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# replacing argparse with defaults
epochs = 200
lr = 0.01
weight_decay = 5e-4
hidden = 16
dropout = 0.5

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

Loading cora dataset...


In [6]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()


    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [10]:
# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 0.3203 acc_train: 0.9500 loss_val: 0.7213 acc_val: 0.7867 time: 0.0263s
Epoch: 0002 loss_train: 0.3063 acc_train: 0.9714 loss_val: 0.7270 acc_val: 0.7800 time: 0.0192s
Epoch: 0003 loss_train: 0.3508 acc_train: 0.9571 loss_val: 0.7763 acc_val: 0.7767 time: 0.0162s
Epoch: 0004 loss_train: 0.3323 acc_train: 0.9357 loss_val: 0.7639 acc_val: 0.7700 time: 0.0126s
Epoch: 0005 loss_train: 0.3620 acc_train: 0.9500 loss_val: 0.7130 acc_val: 0.8033 time: 0.0118s
Epoch: 0006 loss_train: 0.3083 acc_train: 0.9643 loss_val: 0.7581 acc_val: 0.7867 time: 0.0096s
Epoch: 0007 loss_train: 0.3424 acc_train: 0.9786 loss_val: 0.7552 acc_val: 0.7800 time: 0.0096s
Epoch: 0008 loss_train: 0.3332 acc_train: 0.9643 loss_val: 0.7637 acc_val: 0.7867 time: 0.0097s
Epoch: 0009 loss_train: 0.3254 acc_train: 0.9714 loss_val: 0.7295 acc_val: 0.7767 time: 0.0096s
Epoch: 0010 loss_train: 0.3271 acc_train: 0.9500 loss_val: 0.8064 acc_val: 0.7667 time: 0.0102s
Epoch: 0011 loss_train: 0.3334 acc_train

Epoch: 0100 loss_train: 0.3018 acc_train: 0.9643 loss_val: 0.7368 acc_val: 0.7900 time: 0.0138s
Epoch: 0101 loss_train: 0.2722 acc_train: 0.9571 loss_val: 0.7347 acc_val: 0.7767 time: 0.0171s
Epoch: 0102 loss_train: 0.2577 acc_train: 0.9714 loss_val: 0.7020 acc_val: 0.7967 time: 0.0120s
Epoch: 0103 loss_train: 0.2807 acc_train: 0.9571 loss_val: 0.7638 acc_val: 0.7533 time: 0.0110s
Epoch: 0104 loss_train: 0.3027 acc_train: 0.9643 loss_val: 0.7779 acc_val: 0.7600 time: 0.0099s
Epoch: 0105 loss_train: 0.2867 acc_train: 0.9714 loss_val: 0.7142 acc_val: 0.7933 time: 0.0097s
Epoch: 0106 loss_train: 0.2770 acc_train: 0.9786 loss_val: 0.7045 acc_val: 0.7800 time: 0.0102s
Epoch: 0107 loss_train: 0.2720 acc_train: 0.9857 loss_val: 0.7575 acc_val: 0.7800 time: 0.0096s
Epoch: 0108 loss_train: 0.2630 acc_train: 0.9571 loss_val: 0.7100 acc_val: 0.7700 time: 0.0097s
Epoch: 0109 loss_train: 0.3097 acc_train: 0.9643 loss_val: 0.7689 acc_val: 0.7467 time: 0.0097s
Epoch: 0110 loss_train: 0.2558 acc_train

Epoch: 0196 loss_train: 0.2392 acc_train: 0.9857 loss_val: 0.7034 acc_val: 0.7967 time: 0.0122s
Epoch: 0197 loss_train: 0.2441 acc_train: 0.9643 loss_val: 0.7384 acc_val: 0.7767 time: 0.0122s
Epoch: 0198 loss_train: 0.2361 acc_train: 0.9786 loss_val: 0.7447 acc_val: 0.7767 time: 0.0136s
Epoch: 0199 loss_train: 0.2326 acc_train: 0.9714 loss_val: 0.7125 acc_val: 0.7867 time: 0.0101s
Epoch: 0200 loss_train: 0.2204 acc_train: 0.9714 loss_val: 0.7422 acc_val: 0.7700 time: 0.0111s
Optimization Finished!
Total time elapsed: 2.2060s
Test set results: loss= 0.6175 accuracy= 0.8280


The code below is mine.

In [14]:
for i, m in enumerate(model.parameters()):
    print(f"Parameter {i}: {m}")

Parameter 0: Parameter containing:
tensor([[-0.1123,  0.0660,  0.0483,  ..., -0.1961,  0.0372, -0.0574],
        [-0.0229, -0.0583, -0.1448,  ...,  0.0900,  0.0104, -0.0082],
        [ 0.3076, -0.1665,  0.0559,  ..., -0.1334, -0.1438,  0.3124],
        ...,
        [ 0.2490, -0.1269,  0.1377,  ...,  0.1136, -0.1434,  0.1918],
        [ 0.1527, -0.2491, -0.0172,  ..., -0.0866,  0.0041,  0.2658],
        [-0.0747,  0.0845,  0.0492,  ...,  0.0774,  0.0809, -0.0845]],
       requires_grad=True)
Parameter 1: Parameter containing:
tensor([ 0.0883,  0.1602,  0.1791,  0.1063,  0.0942,  0.1657, -0.0002,  0.0788,
         0.1441,  0.1221,  0.1817,  0.1099,  0.1804,  0.1133,  0.1158,  0.1184],
       requires_grad=True)
Parameter 2: Parameter containing:
tensor([[ 2.5628e+00, -2.5031e+00, -2.4827e+00,  2.5081e+00,  2.3554e+00,
         -1.2427e+00, -2.6722e+00],
        [-1.8962e+00,  2.2970e+00, -1.3407e+00, -2.4984e+00, -2.0550e+00,
          2.4408e+00, -1.6600e+00],
        [-1.6479e+00,  1.2

In [15]:
torch.save(model, "model-0.pt")

/home/aaruran/miniconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/aaruran/miniconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GraphConvolution. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [16]:
model2 = torch.load('model-0.pt')

In [18]:
for m1, m2 in zip(model.parameters(), model2.parameters()):
    if not torch.allclose(m1, m2):
        print('Found two parameters with notable difference, saving/loading failed.')

Now, we know that it is possible to save and reload these GCN models.

In [20]:
params = list(model.parameters())

In [22]:
for p in params:
    print(p.shape)

torch.Size([1433, 16])
torch.Size([16])
torch.Size([16, 7])
torch.Size([7])


In [26]:
cat_params = torch.cat([p.reshape(-1) for p in params])
cat_params.shape

torch.Size([23063])

In [29]:
model.state_dict()

OrderedDict([('gc1.weight',
              tensor([[-0.1123,  0.0660,  0.0483,  ..., -0.1961,  0.0372, -0.0574],
                      [-0.0229, -0.0583, -0.1448,  ...,  0.0900,  0.0104, -0.0082],
                      [ 0.3076, -0.1665,  0.0559,  ..., -0.1334, -0.1438,  0.3124],
                      ...,
                      [ 0.2490, -0.1269,  0.1377,  ...,  0.1136, -0.1434,  0.1918],
                      [ 0.1527, -0.2491, -0.0172,  ..., -0.0866,  0.0041,  0.2658],
                      [-0.0747,  0.0845,  0.0492,  ...,  0.0774,  0.0809, -0.0845]])),
             ('gc1.bias',
              tensor([ 0.0883,  0.1602,  0.1791,  0.1063,  0.0942,  0.1657, -0.0002,  0.0788,
                       0.1441,  0.1221,  0.1817,  0.1099,  0.1804,  0.1133,  0.1158,  0.1184])),
             ('gc2.weight',
              tensor([[ 2.5628e+00, -2.5031e+00, -2.4827e+00,  2.5081e+00,  2.3554e+00,
                       -1.2427e+00, -2.6722e+00],
                      [-1.8962e+00,  2.2970e+00, -1.340

state_dict() will be a useful method for saving important attributes of the model to file.